## Library import 

In [21]:
from bs4 import BeautifulSoup  
import pandas as pd
from tqdm import tqdm_notebook
import nltk
import re
from urllib.request import urlopen
import requests 
from selenium import webdriver as wd 
from selenium.webdriver.common.keys import Keys 

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## Crawling

In [10]:
domain='https://movie.naver.com'
story=[]
title=[]
genre=[]
for i in tqdm_notebook(range (1,11)):
    url="https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=pnt&date=20191201&page="+str(i)
    html = urlopen(url)
    soup = BeautifulSoup(html,"html.parser")
    titles=soup.find_all('div',class_='tit5')
    hype=[]
    href=[]
    try:
        for each in titles:
                hype=each.find_all('a')
                for link in hype:
                    href.append(link['href'])
        for j in tqdm_notebook(range(len(href))):
                domain='https://movie.naver.com'
                domain=domain+href[j]
                html=urlopen(domain)
                soup=BeautifulSoup(html,"html.parser")
                story.append(soup.find('p',class_="con_tx").get_text())
                title_tag=soup.find('h3',class_='h_movie')
                title.append(title_tag.find('a').get_text())
                genre_tag=soup.find('p')
                genre.append(genre_tag.find('a').get_text())
    except:
        pass
                  

#스토리 정규화 처리
import re

for i in tqdm_notebook(range(len(story))):
    story[i] = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》“”’]','',story[i] )
    story[i] = re.sub('\r\xa0','',story[i] )

<ipython-input-10-1629fa7d9577>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range (1,11)):


<ipython-input-10-1629fa7d9577>:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for j in tqdm_notebook(range(len(href))):


<ipython-input-10-1629fa7d9577>:34: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(story))):


## 데이터 확인

In [23]:
title[0], genre[0]

('그린 북', '드라마')

## DataFrame 저장

In [24]:
movie_df=pd.DataFrame(data={'제목':title,'줄거리':story,'장르':genre})

In [25]:
movie_df.head()

,제목,줄거리,장르
0,그린 북,1962년 미국 입담과 주먹만 믿고 살아가던 토니 발레롱가비고 모텐슨는 교양과 우아...,드라마
1,가버나움,나를 세상에 태어나게 한 부모님을 고소하고 싶어요출생기록조차 없이 살아온 어쩌면 1...,드라마
2,베일리 어게인,귀여운 소년 이든의 단짝 반려견 베일리는 행복한 생을 마감한다하지만 눈을 떠보니 다...,모험
3,아일라,1950년 한국전쟁에 파병된 슐레이만은 칠흑 같은 어둠 속 홀로 남겨진 5살 소녀를...,드라마
4,주전장,일본의 인종차별 문제를 다룬 영상을 올린 후 우익들의 공격 대상이 된 일본계 미국인...,다큐멘터리


In [26]:
movie_df['합침'] = (movie_df['제목']) + movie_df['줄거리'] + (movie_df['장르'])

In [27]:
movie_df['합침'][0]

'그린 북1962년 미국 입담과 주먹만 믿고 살아가던 토니 발레롱가비고 모텐슨는 교양과 우아함 그 자체인천재 피아니스트 돈 셜리마허샬라 알리 박사의 운전기사 면접을 보게 된다백악관에도 초청되는 등 미국 전역에서 콘서트 요청을 받으며 명성을 떨치고 있는 돈 셜리는위험하기로 소문난 미국 남부 투어 공연을 떠나기로 결심하고투어 기간 동안 자신의 보디가드 겸 운전기사로 토니를 고용한다거친 인생을 살아온 토니 발레롱가와 교양과 기품을 지키며 살아온 돈 셜리 박사생각 행동 말투 취향까지 달라도 너무 다른 두 사람은그들을 위한 여행안내서 그린북에 의존해 특별한 남부 투어를 시작하는데드라마'

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
movie_df['합침'] = movie_df['합침'].fillna('')

In [29]:
tfidf_matrix = tfidf.fit_transform(movie_df['합침'])
# overview에 대해서 tf-idf 수행
print(tfidf_matrix.shape)

(360, 21784)


In [30]:
#코사인유사도
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [32]:
indices = pd.Series(movie_df.index, index=movie_df['제목']).drop_duplicates()
print(indices.head())
#영화의 타이틀과 인덱스를 가진 테이블을 만듬
#영화 타이틀을 입력하면 인덱스를 리턴하려고 만듬

제목
그린 북       0
가버나움       1
베일리 어게인    2
아일라        3
주전장        4
dtype: int64


### 추천 영화의 수

In [33]:
movie_rec_num = 30

In [34]:
def get_recomm(title, cosine_sim=cosine_sim):
    choice = []
    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
    idx = indices[title]

    # 모든 영화에 대해서 해당 영화와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아옵니다.
    sim_scores = sim_scores[1:movie_rec_num + 1]

    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]
    
    for i in range(movie_rec_num):
        choice.append(movie_df['제목'][movie_indices[i]])
    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    print('***영화 추천 순위***')
    for i in range(movie_rec_num):
        print(str(i+1) + '순위 : ' + choice[i])
        
    return sim_scores

In [36]:
genre_sim_df = TfidfVectorizer

### 장르 기반  코사인 유사도 추천 시스템을 위한 전처리

In [37]:
movie_df['장르 상수'] = movie_df['장르'].apply(lambda x : (' ').join(x))

In [38]:
movie_df['장르 상수'] = movie_df['장르 상수'].apply(lambda x : x.replace(' ', ''))

In [39]:
movie_df['장르 상수'] = movie_df['장르 상수'].apply(lambda x : x.replace('/', ' '))

In [40]:
print(movie_df['장르 상수'][30])

멜로 로맨스



## 장르 기반 코사인 유사도

In [41]:
count_vector = CountVectorizer(min_df=0, ngram_range=(1, 2))

In [42]:
genre_mat = count_vector.fit_transform(movie_df['장르 상수'])

In [43]:
gerne_c_sim = cosine_similarity(genre_mat, genre_mat)

In [44]:
genre_sim_sorted_ind = gerne_c_sim.argsort()[:, ::-1]

In [45]:
gerne_c_sim.shape

(360, 360)

In [46]:
def find_sim_movie(df, sorted_ind, title_name, top_n=30):
    title_movie = df[df['제목'] == title_name]
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    return df.iloc[similar_indexes], similar_indexes

In [47]:
movie, movie_idx = find_sim_movie(movie_df, genre_sim_sorted_ind, '모노노케 히메', 30)

[[179 193 281  65  63 288 216  58 126 209 299  49 242 302 129 202 199  39
  196 279 113 111 257  91 251  87  86  85  84 101]]


In [48]:
movie[['제목', '장르']]

,제목,장르
179,고 녀석 맛나겠다,애니메이션
193,명탐정 코난: 천공의 난파선,애니메이션
281,하울의 움직이는 성,애니메이션
65,라푼젤,애니메이션
63,드래곤 길들이기,애니메이션
288,짱구는 못말려 극장판: 정면승부! 로봇아빠의 역습,애니메이션
216,하이 스피드! –프리! 스타팅 데이즈,애니메이션
58,아이언 자이언트,애니메이션
126,이웃집 토토로,애니메이션
209,스파이더맨: 뉴 유니버스,애니메이션


In [49]:
movie_idx

array([179, 193, 281,  65,  63, 288, 216,  58, 126, 209, 299,  49, 242,
       302, 129, 202, 199,  39, 196, 279, 113, 111, 257,  91, 251,  87,
        86,  85,  84, 101], dtype=int64)

In [50]:
story_idx

[(91, 0.05789423623009831),
 (110, 0.049764035708474035),
 (341, 0.045801159678433526),
 (145, 0.04422738729867952),
 (334, 0.043084131772754344),
 (173, 0.040047817988804156),
 (18, 0.03918471528568475),
 (157, 0.03696597905421905),
 (70, 0.03695513841892422),
 (270, 0.03575722708605544),
 (138, 0.035587186263818224),
 (47, 0.03554199617462622),
 (42, 0.03529601263992653),
 (229, 0.03469337267238399),
 (298, 0.03435594064125684),
 (152, 0.032363670718655965),
 (280, 0.03209060349165934),
 (48, 0.03100272452559297),
 (337, 0.03075861672203851),
 (30, 0.03060788376753946),
 (264, 0.02982366368595319),
 (86, 0.02981251443681494),
 (94, 0.02935502727218175),
 (204, 0.02935133948723902),
 (275, 0.02866982151099471),
 (104, 0.028572685047846387),
 (60, 0.02855370216590311),
 (208, 0.028103264303094157),
 (258, 0.02790797004123544),
 (3, 0.027900058825155245)]

### 겹치는 영화 존재하는지 검사

In [82]:
cnt = 0
same_movie = []
k = 0
recommended_movie_genre = list(movie_idx)
print(recommended_movie_genre)
recommended_movie_Tf = list(story_idx)
print(recommended_movie_Tf)
for i in range(movie_rec_num):
    for j in range(movie_rec_num):
        if recommended_movie_Tf[i][0] == recommended_movie_genre[j]:
            cnt += 1
            same_movie.append(recommended_movie_genre[j])

print("test" , recommended_movie_Tf[0][0])
print("겹치는 영화의 수 : ", cnt)
print("겹치는 영화목록 : ", same_movie)
print()

print('***영화 강추 순위***')
for i in same_movie:
    print("영화 강추 " + str(k+1) + "순위: ", title[i])
    k += 1
    
story_idx = get_recomm('모노노케 히메')


        

[179, 193, 281, 65, 63, 288, 216, 58, 126, 209, 299, 49, 242, 302, 129, 202, 199, 39, 196, 279, 113, 111, 257, 91, 251, 87, 86, 85, 84, 101]
[(91, 0.05789423623009831), (110, 0.049764035708474035), (341, 0.045801159678433526), (145, 0.04422738729867952), (334, 0.043084131772754344), (173, 0.040047817988804156), (18, 0.03918471528568475), (157, 0.03696597905421905), (70, 0.03695513841892422), (270, 0.03575722708605544), (138, 0.035587186263818224), (47, 0.03554199617462622), (42, 0.03529601263992653), (229, 0.03469337267238399), (298, 0.03435594064125684), (152, 0.032363670718655965), (280, 0.03209060349165934), (48, 0.03100272452559297), (337, 0.03075861672203851), (30, 0.03060788376753946), (264, 0.02982366368595319), (86, 0.02981251443681494), (94, 0.02935502727218175), (204, 0.02935133948723902), (275, 0.02866982151099471), (104, 0.028572685047846387), (60, 0.02855370216590311), (208, 0.028103264303094157), (258, 0.02790797004123544), (3, 0.027900058825155245)]
test 91
겹치는 영화의 수 :  